In [ ]:

%%capture

!pip install flair

In [ ]:
# Prepare data
data_path = "/content/drive/MyDrive/DataENG/"
exp_id = "ENGEXP1"
save_dir = "/content/drive/MyDrive/DataENG/"+exp_id
model_name = 'xlm-roberta-base'
model_name = 'google/muril-base-cased'

lines = open(data_path+"data40.dis",'r').readlines()
labels = open(data_path+"data40.labels",'r').readlines()

In [ ]:
print(lines[:5])
print(labels[:5])

['send uhm send eta via whatsapp\n', 'send te text to heidi and include moms eta\n', 'send a text to um 5555555\n', 'i want to uh i want to send an email to jon with mike bccd\n', 'send a text to shane on his work number number please\n']
['1 1 0 0 0 0\n', '0 1 0 0 0 0 0 0 0\n', '0 0 0 0 1 0\n', '1 1 1 1 0 0 0 0 0 0 0 0 0 0 0\n', '0 0 0 0 0 0 0 0 1 0 0\n']


In [ ]:
import random
temp = list(zip(lines, labels))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
lines, labels = list(res1), list(res2)

train_lines = lines[:int(0.8*len(lines))]
train_labels = labels[:int(0.8*len(labels))]
val_lines = lines[int(0.8*len(lines)):int(0.9*len(lines))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_lines = lines[int(0.9*len(lines)):]
test_labels = labels[int(0.9*len(labels)):]

In [ ]:
test = pd.DataFrame({'Sentence': test_lines, 'tag': test_labels })

In [ ]:
test_labels

In [ ]:
import pandas as pd
sentence_col = []
word_col = []
tag_col = []
for i in range(len(lines)):
  sent = lines[i][:-1]
  label =labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_data = pd.DataFrame()
df_data['Sentence #'] = sentence_col
df_data['Word'] = word_col
df_data['Tag'] = tag_col

In [ ]:
df_data

,Sentence #,Word,Tag
0,Sentence: 1,please,0
1,None,get,0
2,None,prices,0
3,None,from,0
4,None,um,1
...,...,...,...
16121,None,new,0
16122,None,york,0
16123,None,stock,0
16124,None,exchange,0


In [ ]:
print(len(train_lines))
print(len(val_lines))
len(test_lines)

1272
159


159

In [ ]:
import pandas as pd
sentence_col = []
word_col = []
tag_col = []
for i in range(len(train_lines)):
  sent = train_lines[i][:-1]
  label = train_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_train = pd.DataFrame()
df_train['Sentence #'] = sentence_col
df_train['Word'] = word_col
df_train['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(val_lines)):
  sent = val_lines[i][:-1]
  label = val_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_val = pd.DataFrame()
df_val['Sentence #'] = sentence_col
df_val['Word'] = word_col
df_val['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(test_lines)):
  sent = test_lines[i][:-1]
  label = test_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_test = pd.DataFrame()
df_test['Sentence #'] = sentence_col
df_test['Word'] = word_col
df_test['Tag'] = tag_col

In [ ]:
df_train[:10]

,Sentence #,Word,Tag
0,Sentence: 1,can,0
1,None,we,0
2,None,please,0
3,None,make,0
4,None,a,0
5,None,note,1
6,None,note,0
7,None,for,0
8,None,jack,0
9,Sentence: 2,add,0


In [ ]:
df_train =df_train.fillna(method ="ffill")
df_val=df_val.fillna(method ="ffill")
df_test=df_test.fillna(method ="ffill")

In [ ]:
df_train[:10]

,Sentence #,Word,Tag
0,Sentence: 1,can,0
1,Sentence: 1,we,0
2,Sentence: 1,please,0
3,Sentence: 1,make,0
4,Sentence: 1,a,0
5,Sentence: 1,note,1
6,Sentence: 1,note,0
7,Sentence: 1,for,0
8,Sentence: 1,jack,0
9,Sentence: 2,add,0


In [ ]:
df_val[:10]

,Sentence #,Word,Tag
0,Sentence: 1,pay,0
1,Sentence: 1,off,1
2,Sentence: 1,off,0
3,Sentence: 1,my,0
4,Sentence: 1,visa,0
5,Sentence: 1,card,0
6,Sentence: 1,for,0
7,Sentence: 1,me,0
8,Sentence: 2,yet,0
9,Sentence: 2,there,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df_train["Sentence #"] = LabelEncoder().fit_transform(df_train["Sentence #"] )
df_val["Sentence #"] = LabelEncoder().fit_transform(df_val["Sentence #"] )
df_test["Sentence #"] = LabelEncoder().fit_transform(df_test["Sentence #"] )

In [ ]:
df_train[:20]

,Sentence #,Word,Tag
0,Sentence: 1,please,0
1,Sentence: 1,get,0
2,Sentence: 1,prices,0
3,Sentence: 1,from,0
4,Sentence: 1,um,1
5,Sentence: 1,8,0
6,Sentence: 1,am,0
7,Sentence: 2,tell,0
8,Sentence: 2,me,0
9,Sentence: 2,the,0


In [ ]:
df_train.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
df_val.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
df_test.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
df_test[:20]

,sentence_id,words,labels
0,Sentence: 1,could,0
1,Sentence: 1,it,0
2,Sentence: 1,be,0
3,Sentence: 1,a,0
4,Sentence: 1,cause,0
5,Sentence: 1,for,0
6,Sentence: 1,stumbling,0
7,Sentence: 1,others,0
8,Sentence: 2,log,0
9,Sentence: 2,my,0


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args,use_cuda=False)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
model_dir="/content/drive/MyDrive/data/save"
from sklearn.metrics import accuracy_score

In [ ]:
model.train_model(df_train,df_val=df_test,output_dir=model_dir, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/40 [00:00<?, ?it/s]

(40, 0.5365459417924285)

In [ ]:
result, model_outputs, preds_list = model.eval_model(df_test)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:159: UndefinedMetricWarning: F-s

In [ ]:
result

{'eval_loss': 0.11343261003494262,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [ ]:
preds_list

In [ ]:
prediction, model_output = model.predict(test.Sentence)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
model_output

In [ ]:
prediction

In [ ]:
list1 = [[list(dct.values())[0] for dct in inner_list] for inner_list in prediction]
p1_list = [[int(value) for value in inner_list] for inner_list in list1]

In [ ]:
a1_list = [[int(char) for char in string.strip() if char.isdigit()] for string in test_labels]

In [ ]:
result_list = [
    [
        [a_val, b_val] for a_val, b_val in zip(a_list, b_list)
    ] for a_list, b_list in zip(a1_list, p1_list)
]

result_list[2]
# [actual label , predicted label]

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0]]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
a_list = mlb.fit_transform(a1_list)
p_list = mlb.transform(p1_list)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
#print(classification_report(a_list , p_list, target_names=target_names))
print(classification_report(a_list, p_list, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00       159
     class 1       1.00      0.90      0.95       137

   micro avg       1.00      0.95      0.98       296
   macro avg       1.00      0.95      0.97       296
weighted avg       1.00      0.95      0.98       296
 samples avg       1.00      0.96      0.97       296



In [ ]:
accuracy_score(a_list, p_list)

0.9119496855345912

In [ ]:
sent1 = "Add sandwich add sandwich for lunch"
sent2 = "Add 700 calories a 700 calorie protein shake for breakfast"
sent3 = "take note take note of my vitamin consumption"
sent4="Log intake log calorie intake"
sent5="Today Todays is sunday "
sentences = []
sentences.append(sent1)
sentences.append(sent2)
sentences.append(sent3)
sentences.append(sent4)
sentences.append(sent5)

In [ ]:
pred, model_output = model.predict(sentences)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
pred

[[{'Add': '0'},
  {'sandwich': '1'},
  {'add': '0'},
  {'sandwich': '0'},
  {'for': '0'},
  {'lunch': '0'}],
 [{'Add': '0'},
  {'700': '1'},
  {'calories': '0'},
  {'a': '0'},
  {'700': '0'},
  {'calorie': '0'},
  {'protein': '0'},
  {'shake': '0'},
  {'for': '0'},
  {'breakfast': '0'}],
 [{'take': '1'},
  {'note': '1'},
  {'take': '0'},
  {'note': '0'},
  {'of': '0'},
  {'my': '0'},
  {'vitamin': '0'},
  {'consumption': '0'}],
 [{'Log': '0'},
  {'intake': '1'},
  {'log': '0'},
  {'calorie': '0'},
  {'intake': '0'}],
 [{'Today': '1'}, {'Todays': '0'}, {'is': '0'}, {'sunday': '0'}]]